<a href="https://colab.research.google.com/github/veritaem/Project2/blob/master/Kaggle_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#first up is some imports

In [1]:
!pip install category_encoders

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer, PolynomialFeatures
from sklearn.metrics import precision_score, r2_score, mean_squared_error, classification_report, confusion_matrix, mean_absolute_error, recall_score, accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, f_regression, f_classif
import seaborn as sns
import category_encoders as ce

In [0]:
pd.set_option('display.height', 3000)
pd.set_option('display.max_rows', 3000)
pd.set_option('display.max_columns', 3000)
pd.set_option('display.width', 3000)

In [0]:
df1 = pd.read_csv('https://raw.githubusercontent.com/veritaem/Project2/master/train_features.csv')
df2 = pd.read_csv('https://raw.githubusercontent.com/veritaem/Project2/master/train_labels.csv')
testdf = pd.read_csv('https://raw.githubusercontent.com/veritaem/Project2/master/test_features.csv')

#next up is to look at the data

In [5]:
df1.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [6]:
df2.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [7]:
df = df1.join(df2, rsuffix = 'status_group')
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,idstatus_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,69572,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,8776,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,34310,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,67743,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,19728,functional


In [8]:
df.shape

(59400, 42)

In [9]:
df['status_group'].unique()

array(['functional', 'non functional', 'functional needs repair'],
      dtype=object)

In [0]:
df['status_group'] = df['status_group'].replace({'functional': 2, 'non functional': 0, 'functional needs repair': 1})

In [11]:
df.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [0]:
df['subvillage'] = df['subvillage'].replace(np.nan, 'unknown')
df['funder'] = df['funder'].replace(np.nan, 'unknown')
df['installer'] = df['installer'].replace(np.nan, 'unknown')
df['scheme_management'] = df['scheme_management'].replace(np.nan, 'unknown')
df['permit'] = df['permit'].replace(np.nan, 'unknown')

In [0]:
df['permit'] = df['permit'].replace({'True': 2, 'False': 0, 'unknown': 1})

In [14]:
df.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                       0
gps_height                   0
installer                    0
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                   0
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management            0
scheme_name              28166
permit                       0
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [0]:
'''because certain columns seem unimportant, redundant, 
or missing most of their values, 
I will drop them in this cell, to be modified as needed later'''
df2 = df.drop(['scheme_name', 'public_meeting', 'payment_type', 'region', 'idstatus_group', 'recorded_by', 'wpt_name', 'latitude', 'longitude'],axis = 1)

# ok, time for baselines


In [16]:
df2.status_group.mean()

1.1588383838383838

In [17]:
baseline = [df2.status_group.mean()] * len(df2.status_group)  
mean_absolute_error(df2.status_group, baseline)

0.9136374604632181

In [18]:
maj_classification = df2.status_group.mode()
y_pred = np.full(shape=df2['status_group'].shape, fill_value=maj_classification)
recall_score(df2['status_group'], y_pred, average = 'micro')

0.543080808080808

#bins and engineering?

In [19]:
'''I see two values that would really ebnefit from a bin, and those are 
population and construction year, lets do it!'''

'I see two values that would really ebnefit from a bin, and those are \npopulation and construction year, lets do it!'

In [0]:
year_bins = [-1, 1980, 1990, 2000, 2010, 2020]
year_labels = [1, 2, 3, 4, 5]
df2['year_made'] = pd.cut(df2['construction_year'], bins = year_bins, labels = year_labels)

In [0]:
pop_bins = [-1, 10, 20, 100, 250, 1000, 5000, 10000, 100000]
pop_labels = [1, 2, 3, 4, 5, 6, 7, 8]
df2['pop'] = pd.cut(df2['population'], bins = pop_bins, labels = pop_labels)

#X and y, pipe it!

In [0]:
X = df2.drop(['status_group', 'id', 'funder', 'installer', 'subvillage', 'population', 'construction_year', 'ward', 'num_private'], axis = 1)
y = df2['status_group']

In [23]:
X.describe(include = 'all')

,amount_tsh,date_recorded,gps_height,basin,region_code,district_code,lga,scheme_management,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,year_made,pop
count,59400.000000,59400,59400.000000,59400,59400.000000,59400.000000,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400.0,59400.0
unique,NaN,356,NaN,9,NaN,NaN,125,13,2,18,13,7,12,5,7,8,6,5,5,10,7,3,7,6,5.0,8.0
top,NaN,2011-03-15,NaN,Lake Victoria,NaN,NaN,Njombe,VWC,True,gravity,gravity,gravity,vwc,user-group,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,1.0,1.0
freq,NaN,572,NaN,10248,NaN,NaN,2503,36793,41908,26780,26780,26780,40507,52490,25348,50818,50818,33186,33186,17021,17021,45794,28522,34625,26464.0,28690.0
mean,317.650385,NaN,668.297239,NaN,15.297003,5.629747,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,2997.574558,NaN,693.116350,NaN,17.587406,9.633649,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,NaN,-90.000000,NaN,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.000000,NaN,0.000000,NaN,5.000000,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.000000,NaN,369.000000,NaN,12.000000,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,20.000000,NaN,1319.250000,NaN,17.000000,5.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [25]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(44550, 26)
(14850, 26)
(44550,)
(14850,)


In [26]:
print(X_train.dtypes)
print(y_train.dtypes)

amount_tsh                float64
date_recorded              object
gps_height                  int64
basin                      object
region_code                 int64
district_code               int64
lga                        object
scheme_management          object
permit                     object
extraction_type            object
extraction_type_group      object
extraction_type_class      object
management                 object
management_group           object
payment                    object
water_quality              object
quality_group              object
quantity                   object
quantity_group             object
source                     object
source_type                object
source_class               object
waterpoint_type            object
waterpoint_type_group      object
year_made                category
pop                      category
dtype: object
int64


In [0]:
pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True),
    RobustScaler(),
    LogisticRegression()
)

In [28]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
accuracy_score(y_test, y_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.755016835016835

In [29]:
subby = pipeline.predict(testdf)
subby

ValueError: ignored

In [0]:
#param_grid = {
 #   'logisticregression__C': [.01, .1, 1, 10, 100, 1000],
  #  'logisticregression__class_weight':['balanced', None],
   # 'selectkbest__k': range(1, 6)}


#gs = GridSearchCV(pipeline, param_grid=param_grid, cv=3, 
    #              scoring='recall', 
     #             verbose=10)
#gs.fit(X_train, y_train)

In [0]:
#validation_score = gs.best_score_

#print('Cross-Validation Score:', validation_score)

#print('Best estimator:', gs.best_estimator_)

In [0]:
#selector = gs.best_estimator_.named_steps['polynomialfeatures']
#all_names = X_train.columns
#selected_mask = selector.get_feature_names()


#print('Features selected:')
#for name in selected_mask:
#    print(name)

In [0]:
#y_pred = gs.predict(X_test)
#y_pred

In [0]:
#test_score = mean_absolute_error(y_test, y_pred)
#print('Test Score:', test_score)